In [1]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [3]:
import os
import mysql.connector

conn = mysql.connector.connect(host=os.getenv("MYSQL_HOST") , user=os.getenv("MYSQL_USER"), password=os.getenv("MYSQL_PASSWORD"), database=os.getenv("MYSQL_DB"), auth_plugin='mysql_native_password')

cursor = conn.cursor()

In [4]:
# get database name and show

cursor.execute("SELECT DATABASE()")
db_name = cursor.fetchone()[0]
print(f"Connected to database: {db_name}")

Connected to database: aarong


In [5]:
import pandas as pd

def get_annotable_items():
	query = """ 
			SELECT p.id, p.product_name, p.category, p.sub_category, r.details 
			FROM products p 
			JOIN raw_details r ON p.id = r.product_id
			JOIN annotation_status a on p.id=a.product_id
			WHERE a.manual_annotated=0 and a.flagged = 0;
			"""
	
	cursor.execute(query)
	
	rows = cursor.fetchall()
	columns = [i[0] for i in cursor.description]

	df = pd.DataFrame(rows, columns=columns)
	df_normalized = pd.json_normalize(df['details'].apply(eval))
	df = pd.concat([df, df_normalized], axis=1)

	df = df[[
		'id',
		'category',
		'sub_category',
		'title',
		'description',
		'images',
		'specs.Colour',
		'specs.Fabric',
		'specs.Value Addition',
		'specs.Cut /Fit',
		'specs.Collar/Neck',
		'specs.Sleeve',
		'specs.Bottom Colour',
		'specs.Bottom Fabric',
		'specs.Dupatta Colour',
		'specs.Dupatta Fabric'
	]]

	return df

In [6]:
df = get_annotable_items()

print(len(df))

df.sample(n=3)

3645


,id,category,sub_category,title,description,images,specs.Colour,specs.Fabric,specs.Value Addition,specs.Cut /Fit,specs.Collar/Neck,specs.Sleeve,specs.Bottom Colour,specs.Bottom Fabric,specs.Dupatta Colour,specs.Dupatta Fabric
162,193,Saree,Cotton,Purple Embroidered and Printed Cotton Saree,Purple printed cotton saree with embroidery ov...,[https://www.aarong.com/media/catalog/product/...,Purple,Cotton,Machine Embroidery,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115,140,Saree,Cotton,Mustard Yellow Tangail Cotton Baluchari Saree,Mustard yellow Tangail cotton Baluchari saree ...,[https://www.aarong.com/media/catalog/product/...,Yellow,Cotton,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2814,6496,Panjabi,N/A,Salmon Pink Printed Cotton Panjabi,Salmon pink cotton panjabi with green and pink...,[https://www.aarong.com/media/catalog/product/...,Pink,Cotton,Block Print,A-Line,Band Collar,3-Quarter Sleeve,NaN,NaN,NaN,NaN


In [7]:
import os

def resolve_local_image_path(image_url, image_dir):
	# url "https://www.aarong.com/media/catalog/product/0/5/0560000072957_1.jpg?optimize=high&bg-color=255,255,255&fit=bounds&height=667&width=500&canvas=500:667"
	# resolved as f"{image_dir}/0560000072957_1.jpg"
	image_name = image_url.split("/")[-1].split("?")[0]
	return f"{image_dir}/{image_name}"

def resolve_name_from_url(image_url):
	# url "https://www.aarong.com/media/catalog/product/0/5/0560000072957_1.jpg?optimize=high&bg-color=255,255,255&fit=bounds&height=667&width=500&canvas=500:667"
	# resolved as f"{image_dir}/0560000072957_1.jpg"
	image_name = image_url.split("/")[-1].split("?")[0]
	return f"{image_name}"

def get_segmented_images(images_dir, original_image_name):
	# Return the list of files present in image_dir/original_image_name folder
	segmented_images = []
	segmented_image_dir = os.path.join(images_dir, original_image_name)
	
	for s in os.listdir(segmented_image_dir):
		segmented_images.append(f'{original_image_name}/{s}')

	return segmented_images

In [10]:
# shuffle dataframe
df = df.sample(frac=1).reset_index(drop=True)

def get_annotable_item(segmented_images_dir):
	# Get the first item from the DataFrame
	item = df.iloc[0]
	# Drop the first item from the DataFrame
	df.drop(index=item.name, inplace=True)
	data = item.fillna('').to_dict()

	available_images = []

	original_images = [resolve_name_from_url(image_url) for image_url in data['images']]

	for orig_image_name in original_images:
		segmented_images = []
		if os.path.exists(os.path.join(segmented_images_dir, orig_image_name)):
			segmented_images = get_segmented_images(segmented_images_dir, orig_image_name)
		else:
			print(f"Segmented images not found for {orig_image_name}.")

		# available_images.append({
		# 	'original_image_name': orig_image_name,
		# 	'segmented_images': segmented_images
		# })

		# available_images.append({
		# 	'original_image_name': os.path.join("../data/resized", orig_image_name),
		# 	'segmented_images': [os.path.join("../data/segmented", s) for s in segmented_images]
		# })

		available_images.append({
			'original_image_name': f"/images/resized/{orig_image_name}",
			'segmented_images': [f"/images/segmented/{s}" for s in segmented_images]
		})

	data['available_images'] = available_images

	info = {
		'items_in_queue': len(df),
	}

	return data, info
	

def save_annotation(id, annotation):
	# Save the annotation to the database
	query = """
		UPDATE annotation_status 
		SET manual_annotated=1
		WHERE product_id=%s;
	"""
	cursor.execute(query, (id,))

	# Insert into annotations table- id, product_id, original_image_name, annotated_image_name, segment_type, fabric_types, wear_types, colors, textures, value_additions, is_auto_annotation, comment
	query = """
	INSERT INTO annotations (product_id, annotated_image_name, segment_type, fabric_types, wear_types, colors, textures, value_additions, tags, is_manual_reviewed)
	VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
	"""

	cursor.execute(query, (id, annotation['annotated_image_name'], annotation['segment_type'], annotation['fabric_types'], annotation['wear_types'], annotation['colors'], annotation['textures'], annotation['value_additions'], annotation['tags'], '1'))
	
	conn.commit()

def flag_item(id, comment=''):
	# Flag the item
	query = """
		UPDATE annotation_status 
		SET flagged=1, comment=%s
		WHERE product_id=%s;
	"""
	cursor.execute(query, (comment, id))
	conn.commit()

def discard_image(product_id, image_name, image_path, reason):
	# Insert into discarded_images table
	query = """
		INSERT INTO discarded_images (product_id, image_name, image_path, reason)
		VALUES (%s, %s, %s, %s);
	"""
	cursor.execute(query, (product_id, image_name, image_path, reason))
	conn.commit()

In [11]:
get_annotable_item("../data/raw_images/segmented")

({'id': 196,
  'category': 'Saree',
  'sub_category': 'Cotton',
  'title': 'Red Tangail Cotton Nakshi Par Saree',
  'description': ' Red Tangail cotton nakshi par saree with golden weaving. Comes with matching unstitched blouse piece. Blouse shown in the photo is a styling suggestion, it is not a part of the actual product.\nOrigin:\nThis saree is crafted in the district of Tangail, Bangladesh.',
  'images': ['https://www.aarong.com/media/catalog/product/0/5/0560000074072.jpg?optimize=high&bg-color=255,255,255&fit=bounds&height=667&width=500&canvas=500:667',
   'https://www.aarong.com/media/catalog/product/0/5/0560000074072_2.jpg?optimize=high&bg-color=255,255,255&fit=bounds&height=667&width=500&canvas=500:667',
   'https://www.aarong.com/media/catalog/product/0/5/0560000074072_1.jpg?optimize=high&bg-color=255,255,255&fit=bounds&height=667&width=500&canvas=500:667'],
  'specs.Colour': 'Red',
  'specs.Fabric': 'Cotton',
  'specs.Value Addition': '',
  'specs.Cut /Fit': '',
  'specs.Coll

In [13]:
# Simple Flask api endopints to get the annotable items and save the annotation
from flask import Flask, jsonify, request, send_from_directory
from flask_cors import CORS
import json

ORIGINAL_IMAGES_DIR = "../data/raw_images/resized"
SEGMENTED_IMAGES_DIR = "../data/raw_images/segmented"

app = Flask(__name__)
CORS(app)

@app.route('/images/segmented/<path:filename>')
def serve_segmented_image(filename):
    return send_from_directory(SEGMENTED_IMAGES_DIR, filename)

@app.route('/images/resized/<path:filename>')
def serve_resized_image(filename):
    return send_from_directory(ORIGINAL_IMAGES_DIR, filename)


@app.route('/api/get_annotable_item', methods=['GET'])
def get_annotable_item_api():
	item, info = get_annotable_item(SEGMENTED_IMAGES_DIR)
	if item is not None:
		return jsonify({'item': item, 'info': info}), 200
	else:
		return jsonify({'message': 'No more items to annotate'}), 404
	
@app.route('/api/save_annotation', methods=['POST'])
def save_annotation_api():
	data = request.get_json()

	#print(data)

	id = data['id']
	annotation = data['annotation']
	save_annotation(id, annotation)
	return jsonify({'message': 'Annotation saved successfully'})

@app.route('/api/flag_item', methods=['POST'])
def flag_item_api():
	data = request.get_json()
	id = data['id']
	comment = data['comment']

	if not comment:
		comment = ''
		
	flag_item(id, comment)
	return jsonify({'message': 'Item flagged successfully'})

@app.route('/api/discard_image', methods=['POST'])
def discard_image_api():
	data = request.get_json()
	product_id = data['product_id']
	image_name = data['image_name']
	image_path = data['image_path']
	reason = data['reason']
	discard_image(product_id, image_name, image_path, reason)
	return jsonify({'message': 'Image discarded successfully'})


if __name__ == '__main__':
	app.run(host='0.0.0.0', port=5005)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5005
 * Running on http://172.31.29.107:5005
Press CTRL+C to quit
